In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen


In [2]:
# Reading CSV File for first Batch 0-5000

match_logs_list_urls = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/match_logs_list_urls.csv')

match_logs_list_urls

,Unnamed: 0,0
0,0,/en/players/b682dcf6/matchlogs/2014-2015/summa...
1,1,/en/players/3564689a/matchlogs/2000-2001/summa...
2,2,/en/players/9823be92/matchlogs/2018-2019/summa...
3,3,/en/players/d0a1ddcb/matchlogs/2015-2016/summa...
4,4,/en/players/9bb0b9f1/matchlogs/2018-2019/summa...
...,...,...
84954,84954,/en/players/f1f92f1c/matchlogs/2016-2017/summa...
84955,84955,/en/players/ee6208fa/matchlogs/1993-1994/summa...
84956,84956,/en/players/464cbbc9/matchlogs/2005-2006/summa...
84957,84957,/en/players/2ac01901/matchlogs/2002-2003/summa...


In [3]:
# This one works

match_logs_list_urls = list(match_logs_list_urls['0'])
match_logs_list_urls = ['https://fbref.com' + i for i in match_logs_list_urls]

In [4]:
match_logs_list_urls[0]
start = match_logs_list_urls[0].find("players/") + len("players/")
end = match_logs_list_urls[0].find("/matchlogs")
substring = match_logs_list_urls[0][start:end]

substring, match_logs_list_urls[0]

('b682dcf6',
 'https://fbref.com/en/players/b682dcf6/matchlogs/2014-2015/summary/Sokratis-Papastathopoulos-Match-Logs')

In [5]:
# Pull all match_log_lists_x tables. We will convert each list individually WORK IN PROCESS

def create_match_logs_tables(match_logs_list_urls_x):

    df_30_columns = pd.DataFrame([])
    df_39_columns = pd.DataFrame([])

    count = 0

    for player in match_logs_list_urls_x:
        try: # this may fix "HTTP Error 404: Not Found"
            # urlopen(player)

            new_table = pd.read_html(player)[0]
            new_table.columns = new_table.columns.droplevel()
            new_table['name'] = player.split('/')[-1].replace("-Match-Logs", "")
            
            if new_table.shape[1] == 30:
                new_table['FBRefID'] = player[(player.find("players/") + len("players/")):(player.find("/matchlogs"))]
                df_30_columns = df_30_columns.append(new_table, ignore_index=True)
                count += 1
                
                
            if new_table.shape[1] == 39:
                new_table['FBRefID'] = player[(player.find("players/") + len("players/")):(player.find("/matchlogs"))]
                df_39_columns = df_39_columns.append(new_table, ignore_index=True)
                count += 1

            sys.stdout.write("\r{0} percent player urls have just scraped!".format(count / len(match_logs_list_urls_x)*100))
            sys.stdout.flush()
                

        except:
            pass
    

    return df_30_columns, df_39_columns



In [6]:
len(match_logs_list_urls)

84959

In [11]:
# Creating different length data frames
df_30_columns_1, df_39_columns_1 = create_match_logs_tables(match_logs_list_urls[0:5000])

In [10]:
# print(len(df_30_columns_1['FBRefID'].unique()))
# print(len(df_39_columns_1['FBRefID'].unique()))

18
2


In [ ]:
# df_30_columns_1.head()

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,Off,Crs,TklW,Int,OG,PKwon,PKcon,Match Report,name,FBRefID
0,2014-08-13,Wed,DFL-Supercup,DFL-Supercup,Home,W 2–0,Dortmund,Bayern Munich,Y,CB,...,0,0,0,5,0,0,0,Match Report,Sokratis-Papastathopoulos,2b53f7a5
1,2014-08-16,Sat,DFB-Pokal,First round,Away,W 4–1,Dortmund,S'gart Kickers,Y,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,2b53f7a5
2,2014-08-23,Sat,Bundesliga,Matchweek 1,Home,L 0–2,Dortmund,Leverkusen,Y,NaN,...,0,0,2,6,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,2b53f7a5
3,2014-08-29,Fri,Bundesliga,Matchweek 2,Away,W 3–2,Dortmund,Augsburg,Y,NaN,...,0,0,2,0,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,2b53f7a5
4,2014-09-07,Sun,Euro Qualifying,Group stage,Home,L 0–1,gr Greece,ro Romania,Y,CB,...,0,0,0,3,0,0,0,Match Report,Sokratis-Papastathopoulos,2b53f7a5


In [16]:
# export to google drive as df_30_columns_1 and df_columns_1

pd.DataFrame(df_30_columns_1).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/df_30_columns_1.csv', index=False)
pd.DataFrame(df_39_columns_1).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/df_39_columns_1.csv', index = False)

In [ ]:
# print(df_39_columns_1.columns)
# print(df_30_columns_1.columns)


,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,Att,Cmp%,Prog,Carries,Prog,Succ,Att,Match Report,name,FBRefID
0,2014-08-24,Sun,La Liga,Matchweek 1,Away,W 2–0,Villarreal,Levante,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,Moi-Gomez,5a626448
1,2014-08-28,Thu,Europa Lg,Play-off round,Home,W 4–0,es Villarreal,kz FC Astana,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,Moi-Gomez,5a626448
2,2014-08-31,Sun,La Liga,Matchweek 2,Home,L 0–1,Villarreal,Barcelona,N,"On matchday squad, but did not play",...,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Match Report,Moi-Gomez,5a626448
3,2014-09-14,Sun,La Liga,Matchweek 3,Away,D 0–0,Villarreal,Granada,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,Moi-Gomez,5a626448
4,2014-09-18,Thu,Europa Lg,Group stage,Away,D 1–1,es Villarreal,de M'Gladbach,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,Moi-Gomez,5a626448
5,2014-09-21,Sun,La Liga,Matchweek 4,Home,W 4–2,Villarreal,Rayo Vallecano,N,"On matchday squad, but did not play",...,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Match Report,Moi-Gomez,5a626448
6,2014-09-24,Wed,La Liga,Matchweek 5,Away,D 1–1,Villarreal,Eibar,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,Moi-Gomez,5a626448
7,2014-09-27,Sat,La Liga,Matchweek 6,Home,L 0–2,Villarreal,Real Madrid,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,Moi-Gomez,5a626448
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moi-Gomez,5a626448
9,2014-10-05,Sun,La Liga,Matchweek 7,Away,W 3–1,Villarreal,Celta Vigo,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match Report,Moi-Gomez,5a626448


In [ ]:
#Combining Df_30_columns_1 and df_39_columns_1 to dataframe_1

cols = ['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Match Report', 'Int', 'name', 'FBRefID']

df1 = df_39_columns_1
df2 = df_30_columns_1

df_final_1 = df1.merge(df2,how='outer', left_on=cols, right_on=cols)

print(df1.shape)
print(df2.shape)
# print(df_final.shape)



(56, 40)
(131548, 31)


NameError: name 'df_final' is not defined

In [ ]:
# df_final_1.columns

Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA',
       'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'Prog', 'Carries', 'Prog', 'Succ',
       'Att', 'Match Report', 'name', 'FBRefID', 'Fls', 'Fld', 'Off', 'Crs',
       'TklW', 'OG', 'PKwon', 'PKcon'],
      dtype='object')

In [ ]:
# export to google drive as dataframe_1

pd.DataFrame(df_final_1).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/df_final_1.csv', index=False)

In [21]:
df_final_1

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,name,FBRefID,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon
0,2014-08-24,Sun,La Liga,Matchweek 1,Away,W 2–0,Villarreal,Levante,N,NaN,...,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-08-28,Thu,Europa Lg,Play-off round,Home,W 4–0,es Villarreal,kz FC Astana,Y,NaN,...,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-08-31,Sun,La Liga,Matchweek 2,Home,L 0–1,Villarreal,Barcelona,N,"On matchday squad, but did not play",...,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-09-14,Sun,La Liga,Matchweek 3,Away,D 0–0,Villarreal,Granada,Y,NaN,...,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-09-18,Thu,Europa Lg,Group stage,Away,D 1–1,es Villarreal,de M'Gladbach,Y,NaN,...,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131599,2017-04-23,Sun,3. Liga,Matchweek 34,Away,L 1–2,Fortuna Köln,Zwickau,Y,RM,...,Cauly-Oliveira-Souza,2b53f7a5,0,0,0,0,0,0,0,0
131600,2017-04-28,Fri,3. Liga,Matchweek 35,Home,L 0–1,Fortuna Köln,Paderborn 07,N,NaN,...,Cauly-Oliveira-Souza,2b53f7a5,0,0,0,0,0,0,0,0
131601,2017-05-06,Sat,3. Liga,Matchweek 36,Away,W 2–1,Fortuna Köln,Osnabrück,Y,DM,...,Cauly-Oliveira-Souza,2b53f7a5,0,0,0,0,0,0,0,0
131602,2017-05-20,Sat,3. Liga,Matchweek 38,Away,L 0–4,Fortuna Köln,Mainz 05 II,Y,AM,...,Cauly-Oliveira-Souza,2b53f7a5,0,0,0,0,0,0,0,0
